w_04_05 | BeautifulSoup y HTML
====

BeautifulSoup es, junto con Scrapy, el paquete más usado para scrapping web. Su uso básico tiene una lógica muy simple que consiste en proveerle el código fuente de una página (el HTML) para que construya un "árbol" con sus elementos en donde podemos buscar lo que necesitamos.

Busquemos todas las noticias de la portada de Montevideo Portal, para lo que hay que inspeccionar la fuente.

<img src="files\beautifulsoup_montevideo.png">

In [ ]:
from pprint import pprint

import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.montevideo.com.uy/categoria/Noticias-310"
r = requests.get(url)
soup = BeautifulSoup(r.content)
find = soup.find_all("h2", class_="title")
results = {el.contents[0].text: el.contents[0].get("href") for el in find}
pprint(results, indent=4)

Hicimos lo siguiente:

1. Un request para obtener el HTML.
2. Pasamos el contenido de la respuesta a la clase ``BeautifulSoup`` (se llama igual que el paquete, pero se importa de ``bs4``), lo que devuelve un objeto que podemos procesar fácilmente.
3. Usamos el método ``find_all()`` para que encuentre todos los elementos con el tag h2 y la class "title", porque eso es lo que figura cuando hacemos inspect en un browser.
4. El texto y el link están un nivel más abajo del tag h2, por lo que tenemos que acceder a los ``contents`` (es una lista, por eso accedemos al elemento 0, el único que hay en este caso).
5. Armamos una dict comprehension que extrae el texto y el link (href) de cada elemento encontrado.

¿Por qué no buscamos directamente los tags a (que son links)? Porque necesitamos alguna forma de "filtrar" los resultados, y como ese tag no tiene demasiados detalles, recurrimos a su parent h2. Sino, pasa lo siguiente.

In [ ]:
soup.find_all("a")[:10]

Con el método ``find_all()`` y un poco de creatividad se puede hacer prácticamente todo.

La idea ahora es extraer avisos de puestos de trabajo de El Gallito. En la URL de búsqueda hay información limitada, por lo que habrá que entrar uno por uno, para lo cual necesitamos extraer las URLs.

Vamos a concentranos en los resultados de la primera página.

El proceso es igual, inspeccionamos el elemento que nos interesa, ubicamos los tags relevantes y extraemos.

In [ ]:
gallito = "https://trabajo.gallito.com.uy/buscar"
r = requests.get(gallito)
soup = BeautifulSoup(r.content)
find = soup.find_all("a", class_="post-cuadro row smB")
urls = [puesto.get("href") for puesto in find]
urls

La tarea ahora es constuir las URLs, hacer el request y luego el scrapping.

La ventaja de HTML es que suele estar todo en el mismo lugar. Entonces con ver la estructura en un puesto podemos saber cómo van a funcionar el resto.

In [ ]:
puestos = []
with requests.Session() as s:
    for url in urls:
        full_url = f"https://trabajo.gallito.com.uy{url}"
        print(f"Procesando {full_url}")
        r = s.get(full_url)
        soup = BeautifulSoup(r.content)
        title = soup.find("div", class_="title-puesto").contents[0].text.strip()
        subtitle = soup.find("div", class_="subtitle-puesto").text.strip()
        since = soup.find("span", class_="time-text").text.strip()
        cantidad = soup.find("div", class_="span-ofertas").text.strip()
        titulo_textos = soup.find_all("div", class_="cuadro-aviso-title")
        textos = soup.find_all("div", class_="cuadro-aviso-text")
        textos = {k.text.strip(): v.text.strip() for k, v in zip(titulo_textos, textos)}
        textos.pop("Avisos similares", None)
        data_puesto = {"Descripción": title, "Empresa": subtitle, "Cantidad de ofertas": cantidad, "Fecha": since}
        data_puesto.update(textos)
        puestos.append(data_puesto)

df = pd.DataFrame(puestos)
df.head()

Algunos comentarios:

1. Usamos una ``Session`` de ``requests`` para agilizar el proceso dado que estamos entrando a varias páginas del mismo sitio.
2. Para el título tenemos que revisar los ``contents``, porque es un h2 dentro de un div. No podemos buscar directamente los h2 porque hay otros títulos de ese tipo en la página.
3. El subtítulo (que tiene la información de la empresa), la cantidad de ofertas y el momento de publicación no tienen dificultades, simplemente accedemos a su ``text`` y le aplicamos el método ``strip()`` para que saque todos los espacios extra.
4. Cada puesto tiene distintos boxes con información que no están presentes en todos los avisos. Sin embargo, siempre tienen los mismos tags. Por eso  hacemos un ``find_all()`` para esos tags y luego lo procesamos con una dict comprehension.
5. Algunos avisos tienen un box de "Avisos similares" que no nos interesa por ahora, así que lo eliminamos del diccionario de textos.
6. Construimos un nuevo diccionario con el título, subtítulo, cantidad de ofertas y momento de publicación.
7. Actualizamos este diccionario con la información de los textos.
8. Agregamos el diccionario completo a la lista de ``puestos``